In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np

# Initialize MediaPipe FaceMesh and Drawing utilities
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Specify directory with known faces
KNOWN_FACES_DIR = 'known_faces'
known_face_embeddings = []
known_face_names = []

# Function to calculate face embeddings using MediaPipe FaceMesh landmarks
def get_face_embedding(image_path):
    image = cv2.imread(image_path)
    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            return None
        # Extract the first (and only) face landmarks
        face_landmarks = results.multi_face_landmarks[0]
        # Convert landmarks to a flat numpy array as an "embedding"
        embedding = np.array([[landmark.x, landmark.y, landmark.z] for landmark in face_landmarks.landmark]).flatten()
        return embedding

# Load and process each known face to compute embeddings
for filename in os.listdir(KNOWN_FACES_DIR):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(KNOWN_FACES_DIR, filename)
        embedding = get_face_embedding(image_path)
        if embedding is not None:
            known_face_embeddings.append(embedding)
            known_face_names.append(os.path.splitext(filename)[0])

# Function to compute Euclidean distance between two embeddings
def compute_distance(embedding1, embedding2):
    return np.linalg.norm(embedding1 - embedding2)

# Threshold for recognizing a face (tuned based on trial and error)
THRESHOLD = 0.9

# For webcam input
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1) as face_mesh:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Convert the frame to RGB for processing
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Get the embedding of the detected face
                embedding = np.array([[landmark.x, landmark.y, landmark.z] for landmark in face_landmarks.landmark]).flatten()

                # Initialize best match variables
                best_match_name = "Unknown"
                best_match_distance = float('inf')

                # Compare with each known face
                for i, known_embedding in enumerate(known_face_embeddings):
                    distance = compute_distance(embedding, known_embedding)
                    if distance < best_match_distance and distance < THRESHOLD:
                        best_match_distance = distance
                        best_match_name = known_face_names[i]

                # Display the result on the frame
                frame_height, frame_width, _ = frame.shape
                x_min = int(face_landmarks.landmark[0].x * frame_width)  # Example x-coordinate for positioning text
                y_min = int(face_landmarks.landmark[0].y * frame_height) - 10  # Example y-coordinate for positioning text

                # Draw landmarks and label the face
                mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)
                cv2.putText(frame, best_match_name, (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('MediaPipe Face Recognition (Lightweight)', cv2.flip(frame, 1))

        # Break on ESC key
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


c:\Users\samma\anaconda3\envs\env1\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
